In [1]:
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
import tfr_evaluate
import glob

In [2]:
# add bpnet performance

# add task specific vs 

In [3]:
table_entries = {}

In [4]:
performance_results = pd.concat([pd.read_csv(f) for f in 
                                 ['/mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/model_evaluations/binloss_basenji.csv', 
                                    '/mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/model_evaluations/base_res.csv',
                                   '/mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/model_evaluations/ADDED_base_res.csv',
                                   '/mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/model_evaluations/32_res.csv',
                                    '/mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/model_evaluations/ADDED_bin_loss_40.csv',
                                    '/mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/model_evaluations/bin_loss_40.csv'
                                   ]])

eval_type = 'whole'
pred_type = 'raw'


raw_whole_perf = performance_results[(performance_results['eval type']==eval_type)&
                                    (performance_results['pred type']==pred_type)&
                                   ((performance_results['bin_size']==32)| (performance_results['bin_size']==1))
                                    &(performance_results['loss_fn']=='poisson')]

raw_whole_perf = raw_whole_perf[raw_whole_perf['run_dir']!='paper_runs/new_models/base_res/run-20211022_222400-1pt92q45']
descriptions = raw_whole_perf.iloc[:,7:]
avg_perf = raw_whole_perf.groupby('run_dir').mean().reset_index()

avg_perf_complete = avg_perf.merge(descriptions, how='left', left_on='run_dir',right_on='run_dir').drop_duplicates(subset='run_dir')



In [5]:
for g, df in avg_perf_complete[['model_fn', 'bin_size_x', 'pr_corr', 'activation', 'run_dir']].groupby('bin_size_x'):
    for i, row in (df[['model_fn', 'pr_corr', 'activation', 'run_dir']].sort_values('pr_corr')).iterrows():
#         if row.values[3] == 'paper_runs/basenji/base_res/run-20211022_222400-1pt92q45':
        if row['activation'] == row['activation']:
        
            print('{} ({})'.format(row.values[0], row['activation']),  np.round(row.values[1], 4))
        else:
            print(row.values[0], np.round(row.values[1], 4))

basenji_w1_b64 0.5768
conv_profile_task_base (exponential) 0.5988
bpnet 0.6008
conv_profile_task_base (relu) 0.607
residual_profile_task_base (relu) 0.6369
residual_profile_all_base (relu) 0.6425
residual_profile_task_base (exponential) 0.6536
residual_profile_all_base (exponential) 0.6552
conv_profile_all_dense_32 (exponential) 0.5416
conv_profile_all_dense_32 (relu) 0.5468
bpnet 0.583
basenjimod 0.5845
conv_profile_task_conv_32 (exponential) 0.5964
conv_profile_task_base (exponential) 0.6014
conv_profile_task_base (relu) 0.604
conv_profile_task_conv_32 (relu) 0.6188
residual_profile_all_dense_32 (relu) 0.6548
residual_profile_all_dense_32 (exponential) 0.6652
residual_profile_task_conv_32 (relu) 0.6699
residual_profile_task_conv_32 (exponential) 0.6766


In [6]:
testset, targets, target_dataset_idr = tfr_evaluate.collect_datasets()

In [7]:
bin_size = 32
filepath = 'paper_tables/32_res_eval_bins.csv'

In [10]:
!rm paper_tables/32_res_eval_bins.csv

In [11]:
if not os.path.isfile(filepath):
    best_base_model = avg_perf_complete[avg_perf_complete['bin_size_x']==bin_size].sort_values(['pr_corr']).iloc[-1,0]

    # evaluate at all bin sizes 
    model, raw_bin_size = tfr_evaluate.read_model(best_base_model)
    all_true, all_pred = tfr_evaluate.get_true_pred(model, raw_bin_size, testset)


    bin_sizes = [1, 32, 64, 128, 256, 512, 1024, 2048]
    performance_per_resolution = []
    for eval_bin_size in bin_sizes:
        if eval_bin_size>=raw_bin_size:
            print(raw_bin_size, '--->', eval_bin_size)
            true_for_eval = tfr_evaluate.change_resolution(all_true, raw_bin_size, eval_bin_size)
            pred_for_eval = tfr_evaluate.change_resolution(all_pred, raw_bin_size, eval_bin_size)
            performance = tfr_evaluate.get_performance(true_for_eval, pred_for_eval, targets, 'whole')
            performance_per_resolution.append([raw_bin_size, eval_bin_size] + list(performance.mean().values))

    sorted_personr = pd.DataFrame(performance_per_resolution, columns=['train', 'eval']+list(performance.columns[:-1].values)).sort_values(['train', 'eval'])[['train', 'eval', 'pr_corr']]
    sorted_personr['run_dir'] = best_base_model
    sorted_personr.to_csv(filepath, index=None)
else:
    sorted_personr = pd.read_csv(filepath, index_col=None)
    

32 ---> 32
32 ---> 64
32 ---> 128
32 ---> 256
32 ---> 512
32 ---> 1024


/home/shush/tf_2/lib/python3.7/site-packages/scipy/spatial/distance.py:1300: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(js / 2.0)


32 ---> 2048


In [12]:
for _,row in (sorted_personr).iterrows():
    print(int(row['eval']), np.round(row['pr_corr'], 3))

32 0.677
64 0.679
128 0.684
256 0.692
512 0.7
1024 0.707
2048 0.704
